In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
from sklearn.model_selection import GridSearchCV
import time
from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import (
    accuracy_score, f1_score, confusion_matrix, ConfusionMatrixDisplay, make_scorer
)
from sklearn.feature_selection import RFE
from Preprocessing_functions2 import *

# pandas max columns display
pd.set_option('display.max_columns', None)

## Import Dataset

In [2]:
train_data = pd.read_csv('train_data.csv', index_col='Claim Identifier')
test_data = pd.read_csv('test_data.csv', index_col='Claim Identifier')

C:\Users\inesm\AppData\Local\Temp\ipykernel_8780\3470921380.py:1: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv('train_data.csv', index_col='Claim Identifier')


In [3]:
train_data = train_data[~(train_data.drop(columns=['Assembly Date']).isna().all(axis=1) & train_data['Assembly Date'].notna())]

In [4]:
X = train_data.drop(columns=['Claim Injury Type', 'WCB Decision', 'Agreement Reached','OIICS Nature of Injury Description'])
y = train_data['Claim Injury Type']

test_data = test_data.drop(columns=['OIICS Nature of Injury Description'])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

____

# Auxiliary Functions


## Preprocess Functions

In [8]:
CODE_COLUMNS = ['Industry Code', 'WCIO Cause of Injury Code',
       'WCIO Nature of Injury Code', 'WCIO Part Of Body Code']

DESCRIPTION_COLUMNS = ['WCIO Cause of Injury Description','WCIO Nature of Injury Description','WCIO Part Of Body Description','Industry Code Description']

BOOLEAN_COLUMNS = ['Alternative Dispute Resolution', 'Attorney/Representative','COVID-19 Indicator']

date_order = ['Accident Date', 'C-2 Date','C-3 Date','Assembly Date', 'First Hearing Date']

numerical_columns = [
    'Accident Date', 
    'Age at Injury', 
    'Assembly Date', 
    'Average Weekly Wage', 
    'Birth Year', 
    'C-2 Date', 
    'C-3 Date', 
    'First Hearing Date', 
    'IME-4 Count', 
]

outliers_columns = [
    'Accident Date', 
    'Age at Injury', 
    'Assembly Date', 
    'Average Weekly Wage', 
    'Birth Year',
    'IME-4 Count', 
]

categorical_features = ['Alternative Dispute Resolution',
 'Attorney/Representative',
 'Carrier Name',
 'Carrier Type',
 'County of Injury',
 'COVID-19 Indicator',
 'District Name',
 'Gender',
 'Industry Code',
 'Medical Fee Region',
 'WCIO Cause of Injury Code',
 'WCIO Nature of Injury Code',
 'WCIO Part Of Body Code',
 'Zip Code']


columns_to_scale = ['Accident Date',
                'Assembly Date',
                'Average Weekly Wage',
                'Age at Injury',
                'Birth Year', 
                'Number of Dependents',
                'IME-4 Count']

date_columns = ['Accident Date', 'Assembly Date']

columns_to_drop = ['C-2 Date', 'C-3 Date', 'First Hearing Date']

low_cardinality_cols = [col for col in categorical_features if X[col].nunique() < 10]
high_cardinality_cols = [col for col in categorical_features if X[col].nunique() > 10]


In [9]:
def preprocessing_scaling_encoding_dum(X_train, X_val):
    X_train, X_val = type_conversion_categorical(X_train, X_val,categorical_features)
    X_train, X_val = drop_description_columns(X_train, X_val)
    X_train, X_val = convert_to_timestamp(X_train, X_val, date_order)
    X_train, X_val = convert_to_bool(X_train, X_val, col_names=BOOLEAN_COLUMNS)
    X_train, X_val = impute_mean_numerical(X_train, X_val, numerical_columns)
    X_train, X_val = fill_missing_with_mode(X_train, X_val)
    X_train, X_val = feature_creation_has_Cdate(X_train, X_val)
    X_train, X_val = drop_unwanted_columns(X_train, X_val, columns_to_drop)
    X_train, X_val = winsorize_outliers(X_train, X_val, outliers_columns)
    X_train, X_val = scaling_robust(X_train, X_val, columns_to_scale)
    X_train, X_val = encoding_onehot(X_train, X_val, low_cardinality_cols)
    X_train, X_val = encoding_frequency1(X_train, X_val, high_cardinality_cols)

    return X_train, X_val

In [10]:
def preprocessing_newFeatures_advanced(X_train, X_val):

    # Type conversion
    X_train, X_val = type_conversion_categorical(X_train, X_val, categorical_features)
    X_train, X_val = convert_to_timestamp(X_train, X_val, date_order)
    X_train, X_val = convert_to_bool(X_train, X_val, col_names=BOOLEAN_COLUMNS)

    # Knowledge-based imputation of features
    X_train, X_val = fill_missing_codes_description_based(X_train, X_val)
    X_train, X_val = fillna_zip_code(X_train, X_val)
    X_train, X_val = fillnan_accident_date(X_train, X_val)
    X_train, X_val = fillnan_birth_year(X_train, X_val)
    X_train, X_val = impute_weekly_wage_with_zipIndustryCode(X_train, X_val)

    # Feature creation
    X_train, X_val = feature_creation_has_Cdate(X_train, X_val)
    X_train, X_val = drop_unwanted_columns(X_train, X_val, columns_to_drop)
    X_train, X_val = newFeature_hasIME4(X_train, X_val)
    X_train, X_val = convert_to_datetime(X_train, X_val, date_columns)
    #print dtype of birth year


    X_train, X_val, c_features, n_features= create_groupingFeatures(X_train, X_val)
    categorical_features.append(c_features)
    numerical_columns.append(n_features)

    # Redo type conversion
    X_train, X_val = type_conversion_categorical(X_train, X_val, categorical_features)
    X_train, X_val = convert_to_timestamp(X_train, X_val, date_columns)

    # Treating outliers
    X_train, X_val = winsorize_outliers(X_train, X_val, outliers_columns)

    # Scaling
    X_train, X_val = scaling_robust(X_train, X_val, categorical_features)
    X_train, X_val = scaling_robust(X_train, X_val, numerical_columns)

    # Ensure categorical_features contains unique column names
    valid_categorical_features = list(set(categorical_features))

    # Verify all columns in categorical_features exist in X_train
    valid_categorical_features = [col for col in valid_categorical_features if col in X_train.columns]
    print(f'Valid categorical features: {valid_categorical_features}')


    low_cardinality_cols = [col for col in valid_categorical_features if X_train[col].nunique() < 15]
    high_cardinality_cols = [col for col in valid_categorical_features if X_train[col].nunique() > 15]

    X_train, X_val = encoding_onehot(X_train, X_val, low_cardinality_cols)
    X_train, X_val = encoding_frequency1(X_train, X_val, high_cardinality_cols)

    return X_train, X_val

## Feature Selection


## RFECV with preprocessing_scaling_encoding_dum

In [ ]:
X_enconded, X_val = preprocessing_scaling_encoding_dum(X, X_val)
y_encoded, y_val = encoding_label(y, y_val) 

c:\Users\isa\Documents\Github\ML_Group36\src\Preprocessing_functions2.py:388: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_train[col].fillna(mean_value, inplace=True)
c:\Users\isa\Documents\Github\ML_Group36\src\Preprocessing_functions2.py:389: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves 

In [37]:
 # Initialize RandomForest model
rf_model = RandomForestClassifier(random_state=42)

# Set up cross-validation strategy
cv_strategy = StratifiedKFold(n_splits=5)

# Set up RFECV with RandomForest and cross-validation
rfecv = RFECV(estimator=rf_model, step=1, cv=cv_strategy, scoring='f1_macro') 

# Fit RFECV
rfecv.fit(X_enconded, y_encoded)

#Get the selected features
selected_features_RF = X_enconded.columns[rfecv.support_].tolist()
optimal_num_features = rfecv.n_features_

print("Optimal number of features:", optimal_num_features)
print("Selected Features:", selected_features_RF)

Optimal number of features: 46
Selected Features: ['Accident Date', 'Age at Injury', 'Assembly Date', 'Average Weekly Wage', 'Birth Year', 'Number of Dependents', 'Has First Hearing Date', 'Alternative Dispute Resolution_False', 'Alternative Dispute Resolution_True', 'Alternative Dispute Resolution_nan', 'Attorney/Representative_False', 'Attorney/Representative_True', 'Carrier Type_1A. PRIVATE', 'Carrier Type_2A. SIF', 'Carrier Type_3A. SELF PUBLIC', 'Carrier Type_4A. SELF PRIVATE', 'Carrier Type_5A. SPECIAL FUND - CONS. COMM. (SECT. 25-A)', 'Carrier Type_5C. SPECIAL FUND - POI CARRIER WCB MENANDS', 'Carrier Type_5D. SPECIAL FUND - UNKNOWN', 'Carrier Type_UNKNOWN', 'COVID-19 Indicator_False', 'COVID-19 Indicator_True', 'District Name_ALBANY', 'District Name_BINGHAMTON', 'District Name_BUFFALO', 'District Name_HAUPPAUGE', 'District Name_NYC', 'District Name_ROCHESTER', 'District Name_STATEWIDE', 'District Name_SYRACUSE', 'Gender_F', 'Gender_M', 'Gender_U', 'Gender_X', 'Medical Fee Regio

In [ ]:
selected_features = ['Accident Date', 'Age at Injury', 'Assembly Date', 'Average Weekly Wage', 'Birth Year', 'Number of Dependents', 'Has First Hearing Date', 'Alternative Dispute Resolution_False', 'Alternative Dispute Resolution_True', 'Alternative Dispute Resolution_nan', 'Attorney/Representative_False', 'Attorney/Representative_True', 'Carrier Type_1A. PRIVATE', 'Carrier Type_2A. SIF', 'Carrier Type_3A. SELF PUBLIC', 'Carrier Type_4A. SELF PRIVATE', 'Carrier Type_5A. SPECIAL FUND - CONS. COMM. (SECT. 25-A)', 'Carrier Type_5C. SPECIAL FUND - POI CARRIER WCB MENANDS', 'Carrier Type_5D. SPECIAL FUND - UNKNOWN', 'Carrier Type_UNKNOWN', 'COVID-19 Indicator_False', 'COVID-19 Indicator_True', 'District Name_ALBANY', 'District Name_BINGHAMTON', 'District Name_BUFFALO', 'District Name_HAUPPAUGE', 'District Name_NYC', 'District Name_ROCHESTER', 'District Name_STATEWIDE', 'District Name_SYRACUSE', 'Gender_F', 'Gender_M', 'Gender_U', 'Gender_X', 'Medical Fee Region_I', 'Medical Fee Region_II', 'Medical Fee Region_III', 'Medical Fee Region_IV', 'Medical Fee Region_UK', 'Carrier Name', 'County of Injury', 'Industry Code', 'WCIO Cause of Injury Code', 'WCIO Nature of Injury Code', 'WCIO Part Of Body Code', 'Zip Code']

## RFECV with preprocessing_newFeatures_advanced

In [13]:
X_preprocessed, X_val = preprocessing_newFeatures_advanced(X, X_val)
y_encoded, y_val = encoding_label(y, y_val) 

c:\Users\timst\OneDrive\Desktop\NOVA IMS\Semester 1\MachineLearning\Project\ML_Group36-1\src\Preprocessing_functions2.py:641: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_train['Accident Date'].fillna(X_train['C-2 Date'] - mean_difference_c2_accident, inplace=True)
c:\Users\timst\OneDrive\Desktop\NOVA IMS\Semester 1\MachineLearning\Project\ML_Group36-1\src\Preprocessing_functions2.py:642: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The beh

float64


TypeError: 'int' object is not iterable

In [ ]:
 # Initialize RandomForest model
rf_model = RandomForestClassifier(random_state=42)

# Set up cross-validation strategy
cv_strategy = StratifiedKFold(n_splits=5)

# Set up RFECV with RandomForest and cross-validation
rfecv = RFECV(estimator=rf_model, step=1, cv=cv_strategy, scoring='f1_macro') 

# Fit RFECV
rfecv.fit(X_enconded, y_encoded)

#Get the selected features
selected_features_RF = X_enconded.columns[rfecv.support_].tolist()
optimal_num_features = rfecv.n_features_

print("Optimal number of features:", optimal_num_features)
print("Selected Features:", selected_features_RF)

## GridSearchCV Performance Evaluation


In [16]:
def create_predifined_split(X, y, preprocess_steps,selected_features, n_splits = 5):
    """
    Creates a PredefinedSplit object to be used in cross-validation, more specifically in GridSearchCV.

    Steps:
    - Defines the number of splits
    - Splits the data into training and validation sets
    - Applies the preprocessing steps to the training and validation sets
    - Returns the PredefinedSplit object and the preprocessed data
    """

    X_combined_list = []
    y_combined_list = []

    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    test_data = np.zeros(len(X), dtype=int) - 1

    for fold_idx, (_, test_idx) in enumerate(kf.split(X, y)):
        test_data[test_idx] = fold_idx

    ps = PredefinedSplit(test_fold=test_data)

    for train_index, test_index in ps.split():

        # Get fold
        X_train, X_val = X.iloc[train_index], X.iloc[test_index]
        y_train, y_val = y.iloc[train_index], y.iloc[test_index]

        # Preprocess and encode data    
        X_train, X_val = preprocess_steps(X_train, X_val)
        y_train, y_val = encoding_label(y_train, y_val)

        X_combined_list.append(X_train[selected_features])
        y_combined_list.append(y_train)

    X_combined = pd.concat(X_combined_list, axis=0)
    y_combined = np.concatenate(y_combined_list, axis=0)

    return ps, X_combined, y_combined


        


In [ ]:
def get_best_parameters(X, y, model, param_grid, preprocess_steps, n_splits=5):
    """
    Finds the best hyperparameters for a given model using GridSearchCV.

    Steps:
    - Creates a PredefinedSplit object
    - Creates a GridSearchCV object
    - Fits the GridSearchCV object
    - Returns the best hyperparameters and the best score
    """
    predefined_split, X_combined, y_combined = create_predifined_split(X, y, preprocess_steps,selected_features, n_splits=n_splits)

    scoring = make_scorer(f1_score, average='macro')

    grid_search = GridSearchCV(
        model,
        param_grid,
        cv=predefined_split,
        scoring=scoring,
        n_jobs=-1,
        verbose=1
    )

    grid_search.fit(X_combined, y_combined)

    print("Best Parameters:", grid_search.best_params_)
    print("Best F1-macro Score:", grid_search.best_score_)

    return grid_search.best_params_, grid_search.best_score_

______

# Model Assessment

## Logistic Regression

## Random Forest

## NN

# XGBoost

In [13]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'gamma': [0, 1, 5],
    'reg_alpha': [0, 1, 10],
    'reg_lambda': [1, 10, 100]
}
model=XGBClassifier(random_state=42)

In [ ]:
get_best_parameters(X, y, model, param_grid, preprocessing_scaling_encoding_dum, n_splits=5)

## Knn

In [ ]:
param_grid = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'], # Weight function
    'metric': ['minkowski', 'chebyshev'],  # Distance metrics
    'p': [1, 2], # Power parameter for Minkowski distance
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],  # Algorithm for nearest neighbor search
    'leaf_size': [10, 20, 30, 40, 50] 
}
model=KNeighborsClassifier()

In [ ]:
get_best_parameters(X, y, model, param_grid, preprocessing_scaling_encoding_dum[selected_features], n_splits=5)

## Ensamble Models
look for last year project


# Final Model